In [1]:
#standard
import re
from collections import Counter

#3rd party
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 50
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import miceforest as mf

import catboost

#local
from configs import DATA_PATH

SEED = 1998
np.random.seed(SEED)

In [2]:
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

from itertools import cycle

# Load data

In [3]:
data = pd.read_csv(DATA_PATH, index_col=0)
# data = data.drop(columns="void()")
# print(len(data))
# data = data.dropna()
print(len(data))

11086


In [4]:
# data.columns = [re.sub("\(\)", "", column) for column in data.columns]
# data.columns = [re.sub("\-|\,", "_", column) for column in data.columns]

data[["Activity", "subject"]] = data[["Activity", "subject"]].astype("category")

In [75]:
X, y = data.iloc[:,1:], data.iloc[:,0]
indices = np.random.randint(0, len(X), )
X_sample = X.sample(frac=0.25)
y_sample = y[X_sample.index.tolist()]

In [6]:
pd.DataFrame.from_dict(Counter([col.split('-')[0].split('(')[0] for col in data.columns]), orient='index').rename(columns={0:'count'}).sort_values('count', ascending=False)

,count
fBodyAcc,82
fBodyGyro,79
fBodyAccJerk,79
tBodyGyro,43
tGravityAcc,40
tBodyGyroJerk,40
tBodyAccJerk,40
tBodyAcc,40
tBodyAccMag,13
tBodyAccJerkMag,13


In [7]:
angles = [col for col in X.columns if "angle" in col]

In [10]:
angles

['angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)',
 'shadow-gravity-angle()']

In [11]:
X[angles[:-1]].corrwith(X[angles[-1]])

angle(X,gravityMean)                   -0.070668
angle(Y,gravityMean)                   -0.083565
angle(Z,gravityMean)                    0.118155
angle(tBodyAccJerkMean),gravityMean)   -0.216267
angle(tBodyAccMean,gravity)            -0.109516
angle(tBodyGyroJerkMean,gravityMean)   -0.058102
angle(tBodyGyroMean,gravityMean)       -0.259680
dtype: float64

In [18]:
from sklearn.linear_model import LinearRegression

In [24]:
jerk = "fBodyAccJerk"
acc = "fBodyAcc"
acc_mag = "fBodyAccMag"

In [17]:
jerk_cols = [col for col in X.columns if jerk in col]

In [19]:
acc_cols = [col for col in X.columns if acc in col]

In [25]:
acc_mag = [col for col in X.columns if acc_mag in col]

In [27]:
acc_cols = list(set(acc_cols).difference(jerk_cols).difference(acc_mag))

In [29]:
len(acc_cols)

82

In [31]:
cleaned_jerk = list(map(lambda x: re.sub("Jerk", "", x), jerk_cols))

In [33]:
new = list(set(acc_cols).difference(set(cleaned_jerk)))

In [34]:
new

['fBodyAcc-main()-Y', 'fBodyAcc-main()-Z', 'fBodyAcc-main()-X']

In [35]:
gyro = "tBodyGyro"
acc = "tBodyAcc"

In [36]:
gyro_cols = [col for col in X.columns if gyro in col]
acc_cols = [col for col in X.columns if acc in col]

In [37]:
gyro_cols = list(map(lambda x: re.sub("Gyro", "",x), gyro_cols))
acc_cols = list(map(lambda x: re.sub("Acc", "",x), acc_cols))

In [38]:
new_t = list(set(gyro_cols).difference(acc_cols))

In [40]:
new_t = [col for col in new_t if "angle" not in col]

In [46]:
new_t = list(map(lambda x: re.sub("y-", "yGyro-",x), new_t))
new_t

['tBodyGyro-low()-X', 'tBodyGyro-low()-Z', 'tBodyGyro-low()-Y']

In [71]:
[col for col in X.columns if "energy" in col]

['energy-mean()',
 'fBodyAcc-energy()-X',
 'fBodyAcc-energy()-Y',
 'fBodyAcc-energy()-Z',
 'fBodyAccJerk-energy()-X',
 'fBodyAccJerk-energy()-Y',
 'fBodyAccJerk-energy()-Z',
 'fBodyAccMag-energy()',
 'fBodyBodyAccJerkMag-energy()',
 'fBodyBodyGyroJerkMag-energy()',
 'fBodyBodyGyroMag-energy()',
 'fBodyGyro-energy()-X',
 'fBodyGyro-energy()-Y',
 'fBodyGyro-energy()-Z',
 'tBodyAcc-energy()-X',
 'tBodyAcc-energy()-Y',
 'tBodyAcc-energy()-Z',
 'tBodyAccJerk-energy()-X',
 'tBodyAccJerk-energy()-Y',
 'tBodyAccJerk-energy()-Z',
 'tBodyAccJerkMag-energy()',
 'tBodyAccMag-energy()',
 'tBodyGyro-energy()-X',
 'tBodyGyro-energy()-Y',
 'tBodyGyro-energy()-Z',
 'tBodyGyroJerk-energy()-X',
 'tBodyGyroJerk-energy()-Y',
 'tBodyGyroJerk-energy()-Z',
 'tBodyGyroJerkMag-energy()',
 'tBodyGyroMag-energy()',
 'tGravityAcc-energy()-X',
 'tGravityAcc-energy()-Y',
 'tGravityAcc-energy()-Z',
 'tGravityAccMag-energy()']

In [72]:
drop_cols = new_t+new + ["void()", "shadow-gravity-angle()", "energy-mean()"]

In [73]:
drop_cols

['tBodyGyro-low()-X',
 'tBodyGyro-low()-Z',
 'tBodyGyro-low()-Y',
 'fBodyAcc-main()-Y',
 'fBodyAcc-main()-Z',
 'fBodyAcc-main()-X',
 'void()',
 'shadow-gravity-angle()',
 'energy-mean()']

In [76]:
X.drop(columns=drop_cols, inplace=True)

In [78]:
info = pd.DataFrame.from_dict(Counter([col.split('-')[0].split('(')[0] for col in X.columns]), orient='index').rename(columns={0:'count'}).sort_values('count', ascending=False)
info

,count
fBodyAccJerk,79
fBodyGyro,79
fBodyAcc,79
tBodyAcc,40
tGravityAcc,40
tBodyGyroJerk,40
tBodyGyro,40
tBodyAccJerk,40
tBodyAccMag,13
tBodyGyroMag,13


In [79]:
info["count"].sum()

562

In [80]:
[col for col in X.columns if "gravity" in col]

['angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)']